# ClipCard Agent Test: Clinical/SBAR
**Purpose:** This notebook tests whether an AI agent (LLM) can correctly parse, validate, and reason about ClipCards in the Clinical / SBAR (Situation, Background, Assessment, Recommendation) domain.
It evaluates:1. Can the agent extract kill criteria (stop criteria) from a clinical ClipCard?2. Can it determine if a given patient's vitals (scenario) would trigger the criteria?3. Can it suggest appropriate authority windows for clinical interventions?

In [ ]:
import jsonimport pandas as pd# Load example ClipCardtry:    with open('../../examples/clinical-sbar-test.clipcard.json', 'r') as f:        test_card = json.load(f)    print("Loaded test ClipCard:")    print(json.dumps(test_card, indent=2))except FileNotFoundError:    print("No test ClipCard found.")    test_card = None

### Test 1: Extract Kill Criteria
Can we programmatically parse the clinical stop criteria?

In [ ]:
if test_card:    kill_criteria = test_card.get('kill_criteria', [])    print(f"Found {len(kill_criteria)} kill criteria:\n")    for i, criterion in enumerate(kill_criteria, 1):        print(f"{i}. Condition: {criterion['condition']}")        print(f"   Action: {criterion['action']}\
")else:    print("No ClipCard available for testing.")

### Test 2: Scenario Evaluation
Test if a patient's vitals and lab results would trigger the stop criteria.

In [ ]:
# Define test scenarios for a patientscenarios = [    {        "name": "Stable Patient",        "SpO2": 98,        "HR": 110,        "INR": 1.2,        "expected_trigger": false    },    {        "name": "Hypoxia",        "SpO2": 88,        "HR": 120,        "INR": 1.3,        "expected_trigger": true    },    {        "name": "Tachycardia",        "SpO2": 95,        "HR": 170,        "INR": 1.1,        "expected_trigger": true    },    {        "name": "High INR",        "SpO2": 99,        "HR": 100,        "INR": 4.0,        "expected_trigger": true    }],def evaluate_clinical_scenarios(card, scenarios):    results = []    criteria = card.get('kill_criteria', [])        for scenario in scenarios:        triggered = False        for rule in criteria:            condition = rule['condition']            if 'SpO2' in condition and scenario['SpO2'] < 90:                triggered = True            if 'HR' in condition and scenario['HR'] > 160:                triggered = True            if 'INR' in condition and scenario['INR'] > 3.5:                triggered = True                match = triggered == scenario['expected_trigger']        results.append({            'Scenario': scenario['name'],            'Triggered': triggered,            'Expected': scenario['expected_trigger'],            'Match': '✓' if match else '✗'        })    return pd.DataFrame(results)if test_card:    df_results = evaluate_clinical_scenarios(test_card, scenarios)    print("Evaluating clinical scenarios against the test ClipCard:\n")    print(df_results.to_string(index=False))    print(f"\nTest Accuracy: {df_results['Match'].value_counts().get('✓', 0)}/{len(df_results)}")

### Test 3: Authority Window Recommendations
Generate appropriate authority windows for a new clinical protocol based on risk factors.

In [ ]:
def recommend_clinical_authority_window(impact, uncertainty, reversibility):    """Recommends a clinical-specific authority window based on risk."""    risk_score = impact * uncertainty        if risk_score >= 20 or reversibility >= 5:        return {            "scope_limit": "Single patient, attending supervision only",            "time_limit": "12 hours",            "auto_pause": true,            "justification": "Very high risk requires direct supervision and short duration."        }    elif risk_score >= 15 or reversibility >= 4:        return {            "scope_limit": "Single unit (e.g., PICU), charge nurse oversight",            "time_limit": "72 hours",            "auto_pause": true,            "justification": "High risk requires containing the protocol to one unit with oversight."        }    else:        return {            "scope_limit": "Department-wide, standard protocol",            "time_limit": "14 days post-review",            "auto_pause": false,            "justification": "Lower risk allows for broader rollout after initial review."        }# Test with example clinical scenariostest_cases = [    {"name": "New anticoagulation protocol", "impact": 5, "uncertainty": 4, "reversibility": 4},    {"name": "Adjusting standard saline flush volume", "impact": 2, "uncertainty": 3, "reversibility": 1},    {"name": "Trialing a new type of wound dressing", "impact": 3, "uncertainty": 3, "reversibility": 2}],print("Clinical Authority Window Recommendations:\n")for case in test_cases:    rec = recommend_clinical_authority_window(case['impact'], case['uncertainty'], case['reversibility'])    print(f"{case['name']} (I={case['impact']}, U={case['uncertainty']}, R={case['reversibility']}):")    print(f"  Scope: {rec['scope_limit']}")    print(f"  Duration: {rec['time_limit']}")    print(f"  Auto-pause: {rec['auto_pause']}")    print(f"  Reason: {rec['justification']}\
")